In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
import pandas as pd

from src.subito.scraper import df_maker_subito, next_page_subito
from src.immobiliare.scraper import df_maker_immobiliare, next_page_immobiliare
#from idealista_scraper import next_page_idealista

# City Settings - Affitti

In [8]:
regione = "sardegna"
provincia = "cagliari"
città = "cagliari"

In [9]:
url_immobiliare = f"https://www.immobiliare.it/affitto-case/{città}/"

url_subito = f"https://www.subito.it/annunci-{regione}/affitto/appartamenti/{provincia}/{città}/"

url_idealista = f"https://www.idealista.it/affitto-case/{provincia}-{città}/"

# Immobiliare.it

In [10]:
df_to_concat = [df_maker_immobiliare(url_immobiliare)]
for i in range(2,100):
    print(f"Scraping: run {i} from Immobiliare.it ...")
    df_temp = df_maker_immobiliare(next_page_immobiliare(i, url_immobiliare))
    df_to_concat.append(df_temp)
    if df_temp is None:
        print("\n")
        print("-"*60)
        print("Avaiable pages ended: stopping...")
        print("-"*60)
        break
immobiliare_results = pd.concat(df_to_concat)
n_immobiliare = immobiliare_results.shape[0]

print("\n")
print(f"Numero di appartamenti trovati su Immobiliare.it: {n_immobiliare}")

Scraping: run 2 from Immobiliare.it ...
Scraping: run 3 from Immobiliare.it ...

-->DEBUG ERROR<--
404 Client Error: Not Found for url: https://www.immobiliare.it/affitto-case/cagliari/?pag=3


------------------------------------------------------------
Avaiable pages ended: stopping...
------------------------------------------------------------


Numero di appartamenti trovati su Immobiliare.it: 36


# Idealista

In [11]:
BASE_HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}

# Subito.it

In [12]:
print("Scraping: run 1 from Subito.it ...")
df_to_concat = [df_maker_subito(url_subito)]
for i in range(2,100):
    print(f"Scraping: run {i} from Subito.it ...")
    df_temp = df_maker_subito(next_page_subito(i, url_subito))
    df_to_concat.append(df_temp)
    if df_temp.shape[0] == 0:
        break
print("\n")
print("-"*60)
print("Avaiable pages ended: stopping...")
print("-"*60)
subito_results = pd.concat(df_to_concat)
n_subito = subito_results.shape[0]

print("\n")
print(f"Numero di appartamenti trovati su Subito.it: {n_subito}")

Scraping: run 1 from Subito.it ...
Scraping: run 2 from Subito.it ...
Scraping: run 3 from Subito.it ...
Scraping: run 4 from Subito.it ...
Scraping: run 5 from Subito.it ...
Scraping: run 6 from Subito.it ...
Scraping: run 7 from Subito.it ...


------------------------------------------------------------
Avaiable pages ended: stopping...
------------------------------------------------------------


Numero di appartamenti trovati su Subito.it: 152


# Results

In [13]:
tot_apt = n_subito + n_immobiliare

print("-"*60)
print(f"Totale appartamenti trovati: {tot_apt}")
print("-"*60)

print("\n")
print("Creo Output...")
final_results = pd.concat([subito_results, immobiliare_results])


------------------------------------------------------------
Totale appartamenti trovati: 188
------------------------------------------------------------


Creo Output...


In [16]:
date = str(datetime.today()).split(" ")[0]
final_results.to_excel(f"results_{date}.xlsx")

## Data sperimentation

In [ ]:
#old_data=pd.read_excel("1602.xlsx")
#lista = list(set(subito_results.Link) - set(old_data.Link))
#import webbrowser
#for item in lista:
#    webbrowser.open_new(item.split("//")[1])

In [20]:
test = list(final_results.iloc[0,:])

In [28]:
test

['Grazioso trivano tra mare e città',
 '0',
 'https://www.subito.it/appartamenti/grazioso-trivano-tra-mare-e-citta-cagliari-485571861.htm',
 '3 apr alle 09:44']

# Test Telegram

In [29]:
import requests
TOKEN = "6285394546:AAGn-Z-iwI-jY6RPguUdyBpt16JXGq-tByo"
chat_id = "223987710"

id_francesca = "1667216448"
#message = "hello from your telegram bot"
message = test
url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={chat_id}&text={message}"
print(requests.get(url).json()) # this sends the message

{'ok': True, 'result': {'message_id': 5, 'from': {'id': 6285394546, 'is_bot': True, 'first_name': 'WebScraperAppartamenti', 'username': 'FlatScraperBot'}, 'chat': {'id': 223987710, 'first_name': 'azoto', 'type': 'private'}, 'date': 1681119843, 'text': "['Grazioso trivano tra mare e città', '0', 'https://www.subito.it/appartamenti/grazioso-trivano-tra-mare-e-citta-cagliari-485571861.htm', '3 apr alle 09:44']", 'entities': [{'offset': 44, 'length': 91, 'type': 'url'}]}}


In [ ]:
https://api.telegram.org/bot6285394546:AAGn-Z-iwI-jY6RPguUdyBpt16JXGq-tByo/getUpdates

In [ ]:
{ricerca1: {regione: "regione",
            provincia: "provincia",
            città: "città", }}

 

In [2]:
from telegram.ext import Updater, CommandHandler, MessageHandler

In [4]:
# Funzione chiamata quando viene ricevuto il comando "/start"
def start_command(update, context):
    chat_id = update.message.chat_id
    # Salva la chat ID nel tuo database o in un file
    save_chat_id(chat_id)
    # Puoi anche inviare un messaggio di benvenuto all'utente
    context.bot.send_message(chat_id=chat_id, text="Benvenuto! Grazie per aver avviato il bot.")

def save_chat_id(chat_id):
    # Salva la chat ID nel tuo database o in un file
    # Ad esempio, puoi utilizzare un'operazione di scrittura su file
    with open("chat_ids.txt", "a") as file:
        file.write(str(chat_id) + "\n")

def main():
    # Inizializza il tuo bot
    updater = Updater("6285394546:AAGn-Z-iwI-jY6RPguUdyBpt16JXGq-tByo", update_queue=None)

    # Aggiungi un gestore per il comando "/start"
    updater.dispatcher.add_handler(CommandHandler("start", start_command))

    # Avvia il bot
    updater.start_polling()

    # Mantieni il bot in esecuzione fino a quando non viene interrotto manualmente
    updater.idle()

if __name__ == '__main__':
    main()

AttributeError: 'Updater' object has no attribute 'dispatcher'